In [3]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"]="false"

In [4]:
import functools
from typing import Callable

import jax
from jax.experimental import pallas as pl
# from jax.experimental.pallas import tpu as pltpu
from jax import random
import jax.numpy as jnp
import numpy as np

In [5]:
def matmul_kernel(x_ref, y_ref, z_ref):
  @pl.when(pl.program_id(2) == 0)
  def _():
    z_ref[...] = jnp.zeros_like(z_ref)

  z_ref[...] += x_ref[...] @ y_ref[...]

def matmul(
    x: jax.Array,
    y: jax.Array,
    *,
    bm: int = 128,
    bk: int = 128,
    bn: int = 128,
):
  m, k = x.shape
  _, n = y.shape
  return pl.pallas_call(
      matmul_kernel,
      out_shape=jax.ShapeDtypeStruct((m, n), x.dtype),
      in_specs=[pl.BlockSpec(lambda i, j, k: (i, k), (bm, bk)),
                pl.BlockSpec(lambda i, j, k: (k, j), (bk, bn))],
      out_specs=pl.BlockSpec(lambda i, j, k: (i, j), (bm, bn)),
      grid=(m // bm, n // bn, k // bk)
  )(x, y)

In [6]:
m = k = n = 4096
k1, k2 = random.split(random.key(0), 2)
x = random.normal(k1, (m, k), dtype=jnp.float32)
y = random.normal(k2, (k, n), dtype=jnp.float32)
np.testing.assert_array_equal(x @ y, matmul(x, y))

XlaRuntimeError: RESOURCE_EXHAUSTED: Shared memory size limit exceeded: requested 131072, available: 101376